# Prueba de Dominio, Parte 1
Resolviendo problemas para STATA en Python 3.5


En vista que no tengo Stata, el ejercicio lo haré utilzando python 3.5 como lenguaje base. Existe una integración
con la plataforma que ud. usan (ver aqui: https://www.youtube.com/watch?v=P3_ioddIVKk&feature=youtu.be) por lo que podrás
correr este codigo en el terminal mismo de STATA. En consecuencia, no hay ningún problema de compatibilidad. 



In [1]:
#Voy a utilizar la librería Pandas para transformar datos como en STATA
import pandas as pd 

#(convertir la base en un soloa archivho dta: Padron_web_20190425) 

df1 = pd.read_excel("p_polidocente.xlsx")
df2 = pd.read_excel("p_multigrado.xlsx")
df3 = pd.read_excel("p_eib.xlsx")



####  Antes de concatenar las bases de datos, empezaré con una exploración inicial

In [2]:
#voy a partir por ver que columnas tienen cada una de las bases de datos
df1.columns 

Index(['PLIEGO', 'EJECUTORA', 'DRE/UGEL', 'Código Modular', 'Anexo',
       'Código del local', 'Nombre del Centro Educativo',
       'Tipo de acompañamiento'],
      dtype='object')

In [3]:
df2.columns

Index(['PLIEGO', 'EJECUTORA', 'DRE/UGEL', 'Código Modular', 'Anexo',
       'Código Local', 'Nombre del Centro Educativo', 'Intervención'],
      dtype='object')

In [4]:
df3.columns #la base de datos p_eib tiene una variable menos (intervención/tipo de acompañamiento)

Index(['PLIEGO', 'EJECUTORA', 'DRE/UGEL', 'Código Modular', 'Anexo',
       'Código del local', 'Nombre del Centro Educativo'],
      dtype='object')

In [5]:
df1['Tipo de acompañamiento'].value_counts() #explorando variable Tipo de Acompañamiento

Externo    2388
Interno     823
Name: Tipo de acompañamiento, dtype: int64

In [6]:
df2['Intervención'].value_counts() #explorando variable intervención

ACOMP. MULTIGRADO        6150
EEFF - HC                5455
ACOMP. EIB               3743
Polidocente - Externo    2388
JEC                      2001
REFUERZO                 1483
Polidocente - Interno     823
SPSR                      267
EEFF - HEC                220
CRFA                       71
Name: Intervención, dtype: int64

## Resolviando Parte 1 A.I: Valide que ninguna institución educativa no se encuentre intervenida por más de un acompañamiento, de existir ello eliminarla.

DISCLAIMER 1: No conozco específicamente las nomenclaturas de educación, por lo que estoy asumiendo que aquellos que tienen más de un organismo colocado representan más de un acompañamiento. No me queda claro si los 
acompañamientos externos o multigrado representan más de una institución por lo que no los he incluido en la limpieza de datos.



In [7]:
#He retirado las categorias EEFF-HC y EEFF-HEC ya que entran en mi criterio de más de un acompañante.

df2 = df2[(df2["Intervención"] != 'EEFF - HC') & (df2["Intervención"] != 'EEFF - HEC')] #seleccionando todas las diferentes a esas dos

#Lo que esa linea de código ha hecho es retirar esas categorías de mi base de datos p_multigrado previo a concatenar.

### Ahora, una vez retirado nuestas categorias con más de un acompañamiento, procederé a concatenar las 3 bases de datos. Nuevamente, estoy asumiendo que la variable 'Intervención' y 'Tipo de acompañamiento'; "Código del Local" y "Código Local" hacen referencia a la misma acción, ya que voy a homogenizar sus nombres. 

In [8]:
#Homogenisando nombre de las variables previo a la concatenación (este proceso se realiza una sola vez)

df1["Intervención"] = df1["Tipo de acompañamiento"]
del df1['Tipo de acompañamiento']
df2['Código del local'] = df2['Código Local']
del df2['Código Local']

frames = [df1, df2, df3] #hacer una lista para luego concatenar con facilidad.
result = pd.concat(frames, sort=False) #La variable "Result" ha concatenado nuestra información junta.

In [9]:
#Como resultado tenemos una base de datos con 8 columnas y 23,880 filas.
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23880 entries, 0 to 3742
Data columns (total 8 columns):
PLIEGO                         23880 non-null object
EJECUTORA                      23880 non-null object
DRE/UGEL                       23878 non-null object
Código Modular                 23880 non-null int64
Anexo                          23880 non-null int64
Código del local               23880 non-null int64
Nombre del Centro Educativo    23880 non-null object
Intervención                   20137 non-null object
dtypes: int64(3), object(5)
memory usage: 1.6+ MB


### Respuesta: Aqui abajo con el nombre de result tenemos la base de datos limpia.

In [10]:
result.head(5)

,PLIEGO,EJECUTORA,DRE/UGEL,Código Modular,Anexo,Código del local,Nombre del Centro Educativo,Intervención
0,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,322859,0,346718,7054,Externo
1,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,322875,0,346723,7055 TUPAC AMARU II,Externo
2,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,322891,0,314963,7056 SAN MARTIN DE PORRES,Externo
3,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,322909,0,346737,7057 SOB.ORDEN MILITAR DE MALTA,Externo
4,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,322933,0,329728,7059 JOSE ANTONIO ENCINAS FRANCO,Externo


## Resolviendo Parte 1 A.II: Valide si todas las instituciones propuestas en los padrones se encuentran activas. Señalar en el do si esto no sucede y proceder a eliminarlas

Para realizasr esta sección, voy a utilizar el paquete dbfread para python con el cual leeré el archivo Padrón_web del dropbox.

In [11]:
from dbfread import DBF
from pandas import DataFrame

dbf = DBF('Padron_web.dbf')
frame = DataFrame(iter(dbf)) #Frame es nuestra base de datos del padrón

In [12]:
frame.D_FTE_DATO.value_counts()

Imputado Parcial    111521
No Aplica            55792
Imputado Total        1426
Name: D_FTE_DATO, dtype: int64

In [13]:
frame.D_COD_CAR.value_counts()

No aplica                 122047
Polidocente Completo       14023
Polidocente Multigrado     13883
Unidocente                 10626
No disponible               8160
Name: D_COD_CAR, dtype: int64

### Preparación de Respuesta: como me interesa extraer la variable D_ESTADO, necesito reducir la base de datos del padrón a una expresión más sencilla y hacer un INNER JOIN con nuestra base de datos Result para ver si es que nuestos locales se encuentran activos. Nuestro punto de intercepción entre estas dos bases de datos será el código del local.  

DETALLE IMPORTANTE 2: vamos a incluir aquellas variables que servirán para responder las preguntas de la segunda parte
de este ejercicio.

In [14]:
padron = frame[['CODLOCAL', 'COD_MOD', 'D_ESTADO', 'TALUMNO', 'TDOCENTE']]

In [15]:
padron.head()

,CODLOCAL,COD_MOD,D_ESTADO,TALUMNO,TDOCENTE
0,016100,0415547,Activa,465,17
1,015172,0415638,Activa,494,19
2,015186,0415646,Activa,448,18
3,016751,0415877,Activa,156,9
4,016119,0567206,Activa,60,3


DETALLE IMPORTANTE: he notado que el COD_MOD tiene un digito más que mis variables. Algo 
que posiblemente este relacionado al tipo de dato (Integers vs. Strings). Por ello va a ser necesario tranformar
las variables antes de realizar un merge. 

In [16]:
#Homogenizar nombres con nuestra base de datos result
result['CODLOCAL'] = result['Código del local']
del result['Código del local']
result['COD_MOD'] = result['Código Modular']
del result['Código Modular']

result.head(5)

,PLIEGO,EJECUTORA,DRE/UGEL,Anexo,Nombre del Centro Educativo,Intervención,CODLOCAL,COD_MOD
0,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7054,Externo,346718,322859
1,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7055 TUPAC AMARU II,Externo,346723,322875
2,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7056 SAN MARTIN DE PORRES,Externo,314963,322891
3,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7057 SOB.ORDEN MILITAR DE MALTA,Externo,346737,322909
4,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7059 JOSE ANTONIO ENCINAS FRANCO,Externo,329728,322933


In [17]:
#Agregar COD_MOD un 0

result['COD_MOD'] = result['COD_MOD'].apply('{:0>7}'.format)
result.head(2)

,PLIEGO,EJECUTORA,DRE/UGEL,Anexo,Nombre del Centro Educativo,Intervención,CODLOCAL,COD_MOD
0,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7054,Externo,346718,0322859
1,10. LIMA METROPOLITANA,001. USE 01 SAN JUAN DE MIRAFLORES,UGEL 01 SAN JUAN DE MIRAFLORES,0,7055 TUPAC AMARU II,Externo,346723,0322875


In [18]:
#Transformar datos al tipo object

result.COD_MOD = result.COD_MOD.astype('object')
result.CODLOCAL = result.CODLOCAL.astype('object')
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23880 entries, 0 to 3742
Data columns (total 8 columns):
PLIEGO                         23880 non-null object
EJECUTORA                      23880 non-null object
DRE/UGEL                       23878 non-null object
Anexo                          23880 non-null int64
Nombre del Centro Educativo    23880 non-null object
Intervención                   20137 non-null object
CODLOCAL                       23880 non-null object
COD_MOD                        23880 non-null object
dtypes: int64(1), object(7)
memory usage: 1.6+ MB


### Siendo COD_MOD la variable más especīfica, será el KEY fuente para nuestro JOIN.

In [19]:
data = pd.merge(result, padron, on= 'COD_MOD') 

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24422 entries, 0 to 24421
Data columns (total 12 columns):
PLIEGO                         24422 non-null object
EJECUTORA                      24422 non-null object
DRE/UGEL                       24420 non-null object
Anexo                          24422 non-null int64
Nombre del Centro Educativo    24422 non-null object
Intervención                   20554 non-null object
CODLOCAL_x                     24422 non-null object
COD_MOD                        24422 non-null object
CODLOCAL_y                     24422 non-null object
D_ESTADO                       24422 non-null object
TALUMNO                        24422 non-null int64
TDOCENTE                       24422 non-null int64
dtypes: int64(3), object(9)
memory usage: 2.4+ MB


In [21]:
#Eliminar la variable que se repite: CODLOCAL_y
del data['CODLOCAL_y']

DETALLE IMPORTANTE 3: Se ha eliminado el duplicado CODLOCAL_y luego de haber verificadoq ue representa los mismos valores.

#### Ahora es neesario verificar aquellos que se enceuntran en activo.

In [22]:
data.D_ESTADO.value_counts() #Se estima que 610 casos no se encuentran en activo.

Activa      23812
Inactiva      610
Name: D_ESTADO, dtype: int64

In [23]:
data[data.D_ESTADO == 'Activa'].info() #Dataframe que solo contiene a los activos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23812 entries, 0 to 24421
Data columns (total 11 columns):
PLIEGO                         23812 non-null object
EJECUTORA                      23812 non-null object
DRE/UGEL                       23810 non-null object
Anexo                          23812 non-null int64
Nombre del Centro Educativo    23812 non-null object
Intervención                   20075 non-null object
CODLOCAL_x                     23812 non-null object
COD_MOD                        23812 non-null object
D_ESTADO                       23812 non-null object
TALUMNO                        23812 non-null int64
TDOCENTE                       23812 non-null int64
dtypes: int64(3), object(8)
memory usage: 2.2+ MB


### Parte 1 A. II: Dataframe final validado con activos y Descargada en csv

In [24]:
Padron_web_20190425_Actualizado = data[data.D_ESTADO == 'Activa'] #Cambiar al nombre escogido
Padron_web_20190425_Actualizado.to_csv("Padron_web_20190425_Actualizado.csv") #descargar en formato escogido

## Parte 1 B:  Determine las metas de atención de cada intervención por nivel educativo inicial, primaria y secundaria (número de alumnos, número de docentes y número de IIEE)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24422 entries, 0 to 24421
Data columns (total 11 columns):
PLIEGO                         24422 non-null object
EJECUTORA                      24422 non-null object
DRE/UGEL                       24420 non-null object
Anexo                          24422 non-null int64
Nombre del Centro Educativo    24422 non-null object
Intervención                   20554 non-null object
CODLOCAL_x                     24422 non-null object
COD_MOD                        24422 non-null object
D_ESTADO                       24422 non-null object
TALUMNO                        24422 non-null int64
TDOCENTE                       24422 non-null int64
dtypes: int64(3), object(8)
memory usage: 2.2+ MB


Disclaimer 2: Como es posible ver en el resumen de datos anteriormente expuesto, la variable intervención es la única que cuenta con menos registros. Ello me lleva a tomar una decisión en cuestión de gestión de datos: voy a retirar de mi base de datos todas aquellos casos que tengan a intervención como valor perdido. Esto puede distorsionar los resultados pero al menos nos dirá con mayor precisión que centros educativos recibirán un determinado tipo de internveción.


In [26]:
#Retirando valores perdidos
df = data.dropna(how='any') #Nuestra nueva data final se llama DF
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20552 entries, 0 to 24421
Data columns (total 11 columns):
PLIEGO                         20552 non-null object
EJECUTORA                      20552 non-null object
DRE/UGEL                       20552 non-null object
Anexo                          20552 non-null int64
Nombre del Centro Educativo    20552 non-null object
Intervención                   20552 non-null object
CODLOCAL_x                     20552 non-null object
COD_MOD                        20552 non-null object
D_ESTADO                       20552 non-null object
TALUMNO                        20552 non-null int64
TDOCENTE                       20552 non-null int64
dtypes: int64(3), object(8)
memory usage: 1.9+ MB


In [27]:
"""
Considerando que son posibles varias intervenciones en un mismo centro educativo, vamoa s tener
que hacer un nuevo dataframe considerando el conteo de las IIEE
"""

df_temp = df['Nombre del Centro Educativo'].value_counts().rename_axis('unique_values').reset_index(name='counts')
#La base dee datos df_temp ahora tiene separados los IIEE y el conteo de instituciones
df_temp.head(10) #Mostrar solo 10 como ejemplo

,unique_values,counts
0,JOSE CARLOS MARIATEGUI,50
1,JOSE MARIA ARGUEDAS,32
2,CESAR VALLEJO,29
3,VICTOR RAUL HAYA DE LA TORRE,23
4,JUAN VELASCO ALVARADO,21
5,ANDRES AVELINO CACERES,21
6,RICARDO PALMA,20
7,SIMON BOLIVAR,19
8,SAGRADO CORAZON DE JESUS,18
9,MANUEL GONZALES PRADA,16


In [28]:
total_IIEE = df_temp['unique_values'].count()
total_alumnos = df.TALUMNO.sum()
total_Docentes = df.TDOCENTE.sum()

print('Número Total de IIEE:', total_IIEE)
print("Numero Total de alumnos:", total_alumnos)
print("Numero Total de Docentes:", total_Docentes)

Número Total de IIEE: 14401
Numero Total de alumnos: 4157067
Numero Total de Docentes: 218988


## Parte 1 C: Estime el número de acompañantes considerando los siguientes ratios de acuerdo al ámbito: URBANO: 1 Acompañante por 15 Docentes. RURAL : 1 Acompañante por 8 Docentes 

He econtrado una mejor forma de trabajar los archivos DBF transformandolos directamente a CSV con esta función

In [29]:
#función para usar archivos dbf
import csv
from dbfread import DBF

def dbf_to_csv(dbf_table_pth):#Input a dbf, output a csv, same name, same path, except extension
    csv_fn = dbf_table_pth[:-4]+ ".csv" #Set the csv file name
    table = DBF(dbf_table_pth)# table variable is a DBF object
    with open(csv_fn, 'w', newline = '') as f:# create a csv file, fill it with dbf content
        writer = csv.writer(f)
        writer.writerow(table.field_names)# write the column name
        for record in table:# write the rows
            writer.writerow(list(record.values()))
    return csv_fn# return the csv name

In [30]:
"""
Al usar la función solo necesito colocar el archivo sin utilizar más lienas de codigo.
"""
dbf_to_csv('Padlocaladi_web.dbf') 

'Padlocaladi_web.csv'

In [31]:
padron_local = pd.read_csv("Padlocaladi_web.csv")

In [32]:
padron_local.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 27 columns):
CODLOCAL      95 non-null int64
COD_MOD       95 non-null int64
TIPOLOCAL     95 non-null int64
D_TIPOLOC     95 non-null object
DIRECCION     95 non-null object
LOCALIDAD     0 non-null float64
REFERENCIA    3 non-null object
CODGEO        95 non-null int64
NOM_DPTO      95 non-null object
NOM_PROV      95 non-null object
NOM_DIST      95 non-null object
CODUGEL       95 non-null int64
D_DREUGEL     95 non-null object
REGGER_ED     95 non-null object
CODCP_INEI    95 non-null int64
CODCP_MED     95 non-null int64
CATEG_CP      0 non-null float64
D_CATEG_CP    0 non-null float64
CEN_POB       95 non-null object
COD_AREA      95 non-null int64
D_CODAREA     95 non-null object
NLATITUD      95 non-null float64
NLONGITUD     95 non-null float64
NALT_LOCAL    0 non-null float64
NZOOM         95 non-null int64
STATLOCAL     95 non-null int64
D_STATLOCA    95 non-null object
dtypes: float64

In [33]:
padron_local['CODLOCAL'] = padron_local['CODLOCAL'].astype('object')

In [34]:
padron_local_2 = padron_local[['CODLOCAL','D_CODAREA']]

In [36]:
#df_c = pd.merge(df, padron_local_2,  on='CODLOCAL', how='left') #Esta base de datos contiene el join.

DISCLAIMER 2: la base de datos "padron_local" solo tiene 95 casos de información URBANA por lo que en la práctica no distingue entre urbano y rural. EN TEORIA, de contener la información necesara el JOIN realizado en base al Código Local deberia funcionar. 

In [37]:
padron_local_2.D_CODAREA.value_counts()

URBANA    95
Name: D_CODAREA, dtype: int64

### Respuesta 1 C: Asumiendo que toda la base de datos es urbana, el ratio de acompañamiento sería 1-15. He creado la variable Ratio_Acomp para cada caso siguiendo los criterios expuestos en el còdigo abajo

In [40]:
#será necesario crear un for loop para generar una nueva variable que contabilice el nuevo ratio automaticamente
lista_nueva = [] #nuuestro repositorio para el ratio
for numero in df.TDOCENTE:
    ratio = numero/15
    lista_nueva.append(ratio) #añadir a mi lista el ratio para cada caso
df['Ratio_Acomp'] = lista_nueva  
    
df[['Nombre del Centro Educativo', 'TDOCENTE', 'Ratio_Acomp']].head(10) #Ejemplo con los 10 primeros casos.   

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Nombre del Centro Educativo,TDOCENTE,Ratio_Acomp
0,7054,47,3.133333
1,7054,47,3.133333
2,7054,47,3.133333
3,7055 TUPAC AMARU II,30,2.000000
4,7055 TUPAC AMARU II,30,2.000000
5,7055 TUPAC AMARU II,30,2.000000
6,7056 SAN MARTIN DE PORRES,26,1.733333
7,7056 SAN MARTIN DE PORRES,26,1.733333
8,7056 SAN MARTIN DE PORRES,26,1.733333
9,7057 SOB.ORDEN MILITAR DE MALTA,43,2.866667


In [41]:
df.to_csv("Padron_web_20190425_Actualizado_con_Ratios.csv") #descargar en formato escogido

# Prueba Dominio, Parte 2
Se tiene información presupuestaria de las regiones con los siguientes cortes: 25 de febrero
(archivo: 20190225_GR) y el 31 de marzo (20190331_GR)

In [42]:
data_GR1 = pd.read_excel("20190331_GR.xlsx")

In [43]:
data_GR2 = pd.read_excel("20190225_GR.xlsx")

In [44]:
data_GR1.head(2)

,ANO_EJE,TIPO_ACT_PROY,TIPO_GOBIERNO,SUB_TIPO_GOBIERNO,SECTOR,SECTOR_NOMBRE,PLIEGO,PLIEGO_NOMBRE,EJECUTORA,EJECUTORA_NOMBRE,...,ESPECIFICA_NOMBRE,ESPECIFICA_DET,ESPECIFICA_DET_NOMBRE,MONTO_PIA,MONTO_PIM,MONTO_CERTIFICADO,MONTO_COMPROMETIDO_ANUAL,MONTO_COMPROMETIDO,MONTO_DEVENGADO,MONTO_GIRADO
0,2019,PROYECTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,455,GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA,300,REGION MOQUEGUA-EDUCACION,...,OTRAS INVERSIONES INTANGIBLES,1,GASTOS POR LA CONTRATACION DE PERSONAL,0,70000,27000.0,26995.31,0.0,0.0,0.0
1,2019,PROYECTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,455,GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA,300,REGION MOQUEGUA-EDUCACION,...,OTRAS INVERSIONES INTANGIBLES,2,GASTOS POR LA COMPRA DE BIENES,0,27614,0.0,0.00,0.0,0.0,0.0


In [45]:
data_GR2.head(2)

,ANO_EJE,TIPO_ACT_PROY,TIPO_GOBIERNO,SUB_TIPO_GOBIERNO,SECTOR,SECTOR_NOMBRE,PLIEGO,PLIEGO_NOMBRE,EJECUTORA,EJECUTORA_NOMBRE,...,ESPECIFICA_NOMBRE,ESPECIFICA_DET,ESPECIFICA_DET_NOMBRE,MONTO_PIA,MONTO_PIM,MONTO_CERTIFICADO,MONTO_COMPROMETIDO_ANUAL,MONTO_COMPROMETIDO,MONTO_DEVENGADO,MONTO_GIRADO
0,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,464,GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCION...,300,REGION CALLAO - EDUCACION CALLAO,...,OTROS SERVICIOS,99,SERVICIOS DIVERSOS,53,53,0.0,0.0,0.0,0.0,0.0
1,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,459,GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN MARTIN,300,REGION SAN MARTIN-EDUCACION,...,DE OFICINA,2,"PAPELERIA EN GENERAL, UTILES Y MATERIALES DE O...",500,500,0.0,0.0,0.0,0.0,0.0


### A) Crear una base de datos “ejec_pp” que considere los siguiente: Genérica de gasto: “BIENES Y SERVICIOS”Programas presupuestales (“PROGRAMA_PPTO”): 90, 106, 107, 147 y 91 Fuente de Financiamiento (“FUENTE_FINANC_NOMBRE”): Recursos Ordinarios

In [46]:
#Concatenando las dos bases de datos en una sola llamada ejec_pp

BDS = [data_GR1, data_GR2]

In [47]:
ejec_pp_base = pd.concat(BDS, sort=False)
ejec_pp_base.head(5) #mostrar los 5 primeros de nuestra nueva base de datos

,ANO_EJE,TIPO_ACT_PROY,TIPO_GOBIERNO,SUB_TIPO_GOBIERNO,SECTOR,SECTOR_NOMBRE,PLIEGO,PLIEGO_NOMBRE,EJECUTORA,EJECUTORA_NOMBRE,...,ESPECIFICA_NOMBRE,ESPECIFICA_DET,ESPECIFICA_DET_NOMBRE,MONTO_PIA,MONTO_PIM,MONTO_CERTIFICADO,MONTO_COMPROMETIDO_ANUAL,MONTO_COMPROMETIDO,MONTO_DEVENGADO,MONTO_GIRADO
0,2019,PROYECTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,455,GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA,300,REGION MOQUEGUA-EDUCACION,...,OTRAS INVERSIONES INTANGIBLES,1,GASTOS POR LA CONTRATACION DE PERSONAL,0,70000,27000.0,26995.31,0.0,0.0,0.0
1,2019,PROYECTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,455,GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA,300,REGION MOQUEGUA-EDUCACION,...,OTRAS INVERSIONES INTANGIBLES,2,GASTOS POR LA COMPRA DE BIENES,0,27614,0.0,0.00,0.0,0.0,0.0
2,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,463,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,300,REGION LIMA - EDUCACION LIMA PROVINCIAS,...,DE OFICINA,2,"PAPELERIA EN GENERAL, UTILES Y MATERIALES DE O...",3000,3000,0.0,0.00,0.0,0.0,0.0
3,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,454,GOBIERNO REGIONAL DEL DEPARTAMENTO DE MADRE DE...,300,REGION MADRE DE DIOS-EDUCACION,...,DE OFICINA,2,"PAPELERIA EN GENERAL, UTILES Y MATERIALES DE O...",0,1000,0.0,0.00,0.0,0.0,0.0
4,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,446,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,300,REGION CUSCO-EDUCACION,...,DE OFICINA,2,"PAPELERIA EN GENERAL, UTILES Y MATERIALES DE O...",1000,1000,0.0,0.00,0.0,0.0,0.0


In [48]:
ejec_pp_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117851 entries, 0 to 57396
Data columns (total 58 columns):
ANO_EJE                     117851 non-null int64
TIPO_ACT_PROY               117851 non-null object
TIPO_GOBIERNO               117851 non-null object
SUB_TIPO_GOBIERNO           117851 non-null object
SECTOR                      117851 non-null int64
SECTOR_NOMBRE               117851 non-null object
PLIEGO                      117851 non-null int64
PLIEGO_NOMBRE               117851 non-null object
EJECUTORA                   117851 non-null int64
EJECUTORA_NOMBRE            117851 non-null object
SEC_EJEC                    117851 non-null int64
PROGRAMA_PPTO               117851 non-null int64
PROGRAMA_PPTO_NOMBRE        117851 non-null object
FUNCION                     117851 non-null int64
FUNCION_NOMBRE              117851 non-null object
PROGRAMA                    117851 non-null int64
PROGRAMA_NOMBRE             117851 non-null object
SUB_PROGRAMA                117

### *Respuesta A en 3 pasos:*

In [50]:
#Solo genérica de gastos de Bienes y servicios
paso1 = ejec_pp_base[ejec_pp_base.GENERICA_NOMBRE == 'BIENES Y SERVICIOS']

In [51]:
#incluir Programa_ppto: 90, 106, 107, 147, 91
paso2 = paso1.loc[paso1['PROGRAMA_PPTO'].isin(['90','106', '107', '147', '91'])]

In [52]:
#Incluir solo el tipo Recursos Ordinarios
paso3 = paso2[paso2['FUENTE_FINANC_NOMBRE'] == "RECURSOS ORDINARIOS"]

In [53]:
ejec_pp = paso3
ejec_pp.info() #tenemos una totalidad de 35,634 casos aplicados los filros

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35634 entries, 10 to 57394
Data columns (total 58 columns):
ANO_EJE                     35634 non-null int64
TIPO_ACT_PROY               35634 non-null object
TIPO_GOBIERNO               35634 non-null object
SUB_TIPO_GOBIERNO           35634 non-null object
SECTOR                      35634 non-null int64
SECTOR_NOMBRE               35634 non-null object
PLIEGO                      35634 non-null int64
PLIEGO_NOMBRE               35634 non-null object
EJECUTORA                   35634 non-null int64
EJECUTORA_NOMBRE            35634 non-null object
SEC_EJEC                    35634 non-null int64
PROGRAMA_PPTO               35634 non-null int64
PROGRAMA_PPTO_NOMBRE        35634 non-null object
FUNCION                     35634 non-null int64
FUNCION_NOMBRE              35634 non-null object
PROGRAMA                    35634 non-null int64
PROGRAMA_NOMBRE             35634 non-null object
SUB_PROGRAMA                35634 non-null int64

### *Respuesta B) Con la base “data_GR1”, crea la variable e_marzo que estime cuanto seria el devengado para el mes de marzo.*

In [54]:
#asumiendo que la data relfleja un mes exacto de ejercicio, la sumatoria del devengado reflejaría el monto.
e_marzo = data_GR1.MONTO_DEVENGADO.sum()
print("Develngado del mes de Marzo:", round(e_marzo))

Develngado del mes de Marzo: 3595503184.0


### C) Crear la variable “tipo” y consignar “CAS” para aquellas observaciones de la variable especifica (“ESPECIFICA_DET_NOMBRE”) que contenga la palabra “C.A.S”. Para el resto de observaciones consignar “No CAS”

In [55]:
#en vista de que son varias las asignaciones a una misma categoria voy a tener que hacer algo un poco más complejo
ejec_pp.ESPECIFICA_DET_NOMBRE.value_counts()


TRANSPORTE Y TRASLADO DE CARGA, BIENES Y MATERIALES                                                   4082
VIATICOS Y ASIGNACIONES POR COMISION DE SERVICIO                                                      2558
VACACIONES TRUNCAS DE C.A.S.                                                                          2541
PAPELERIA EN GENERAL, UTILES Y MATERIALES DE OFICINA                                                  2235
MATERIAL DIDACTICO, ACCESORIOS Y UTILES DE ENSEÑANZA                                                  2177
OTROS GASTOS                                                                                          2122
CONTRIBUCIONES A ESSALUD DE C.A.S.                                                                    2119
CONTRATO ADMINISTRATIVO DE SERVICIOS                                                                  2106
AGUINALDOS DE C.A.S.                                                                                  2058
PASAJES Y GASTOS DE TRANSPORTE       

In [56]:
cas = ejec_pp.ESPECIFICA_DET_NOMBRE #solo va a contener la variable que necesitamos
test_set = {'C.A.S', 'C.A.S.'} #nuestro test tiene dos ejemplos de cas diferentes registrados diferente
"""
Las lineas de código son sencillas pero lo que aquí va a hacer es complejo:
voy a crear una función que busque especificamente la secuencia C.A.S y la
transforme en una nueva variable dicotomica

"""
ejec_pp['CAS'] = cas.apply(lambda x: sum(i in test_set for i in x.split()))


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### *Respuesta C: He creado la variable CAS como dicotómica númerica (es decir, 1 si es que es Cas y 0 si no lo es). No creo que sea necesario cambiarlo a un nominal.*

In [57]:
ejec_pp.CAS.value_counts()

0    28908
1     6726
Name: CAS, dtype: int64

### D) Modificar la variable “tipo” consignado “PROPINAS” si en las observaciones de la variable especifica contiene “ANIMADORAS Y ALFABETIZADORES”

In [58]:
ejec_pp[ejec_pp['ESPECIFICA_DET_NOMBRE'] == 'ANIMADORAS Y ALFABETIZADORES' ]

,ANO_EJE,TIPO_ACT_PROY,TIPO_GOBIERNO,SUB_TIPO_GOBIERNO,SECTOR,SECTOR_NOMBRE,PLIEGO,PLIEGO_NOMBRE,EJECUTORA,EJECUTORA_NOMBRE,...,ESPECIFICA_DET,ESPECIFICA_DET_NOMBRE,MONTO_PIA,MONTO_PIM,MONTO_CERTIFICADO,MONTO_COMPROMETIDO_ANUAL,MONTO_COMPROMETIDO,MONTO_DEVENGADO,MONTO_GIRADO,CAS
1898,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,463,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,309,REGION LIMA - EDUCACION BARRANCA,...,4,ANIMADORAS Y ALFABETIZADORES,445000,445000,44700.0,44700.0,44700.00,44700.00,44700.00,0
1950,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,463,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,302,REGION LIMA - EDUCACION HUAURA,...,4,ANIMADORAS Y ALFABETIZADORES,692000,692000,67300.0,67300.0,67300.00,67300.00,67300.00,0
1979,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,462,GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCAYALI,302,GOB. REG. DE UCAYALI - EDUCACION ATALAYA,...,4,ANIMADORAS Y ALFABETIZADORES,622000,622000,39800.0,39800.0,39800.00,39800.00,39800.00,0
2051,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,458,GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO,310,REGION PUNO- EDUCACION SANDIA,...,4,ANIMADORAS Y ALFABETIZADORES,620000,620000,618000.0,61800.0,61800.00,61800.00,61800.00,0
2349,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,452,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LAMBAYEQUE,302,REGION LAMBAYEQUE - EDUCACION LAMBAYEQUE,...,4,ANIMADORAS Y ALFABETIZADORES,2430835,2430835,2430835.0,2430835.0,135100.00,135100.00,135100.00,0
2390,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,451,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA LIBERTAD,313,GOB. REG. DE LA LIBERTAD - EDUCACION EL PORVENIR,...,4,ANIMADORAS Y ALFABETIZADORES,340127,340127,246000.0,246000.0,24600.00,24600.00,24600.00,0
2421,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,451,GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA LIBERTAD,305,REGION LA LIBERTAD-EDUCACION OTUZCO,...,4,ANIMADORAS Y ALFABETIZADORES,408035,408035,280000.0,28000.0,28000.00,28000.00,22400.00,0
2452,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,450,GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,310,GOB. REG. DE JUNIN - EDUCACION PICHANAKI,...,4,ANIMADORAS Y ALFABETIZADORES,179000,179000,179000.0,179000.0,17700.00,17700.00,17700.00,0
2488,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,449,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,304,REGION ICA - EDUCACION PALPA,...,4,ANIMADORAS Y ALFABETIZADORES,170000,170000,168000.0,168000.0,16800.00,16800.00,16800.00,0
2508,2019,ACTIVIDAD / PRODUCTO,GOBIERNOS REGIONALES,R,99,GOBIERNOS REGIONALES,449,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,300,REGION ICA-EDUCACION,...,4,ANIMADORAS Y ALFABETIZADORES,1251000,1251000,1251000.0,1251000.0,126700.00,126700.00,126700.00,0


Hay multiples formar de hacer esto. Por ser la más rápida optaré por crear una nueva variable y replazar TIPO_TRANSACCION_NOMBRE, ya que solo tiene una categoría: GASTO.

In [59]:
#primero voy a crear una nueva lista vacīa para mi variable
lista_nueva = []
#Voy a hacer unn for loop para cada uno de las Especificas Animadoras y Alfabetizadores
for nombre in ejec_pp['ESPECIFICA_DET_NOMBRE']:
    if nombre == 'ANIMADORAS Y ALFABETIZADORES':
        lista_nueva.append('PROPINA')
    else: 
        lista_nueva.append('GASTO')
ejec_pp['TIPO_TRANSACCION_NOMBRE_2'] = lista_nueva
#Lo que esas lineas de codigo ha hecho es decirle a la compu que genere Propina si es que encuentra
#'Animadoras y alfabetizadores' o en su defecto que genere Gasto.

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
#ahora procederé a eliminar TIPO_TRANSACCION_NOMBRE
del ejec_pp['TIPO_TRANSACCION_NOMBRE']

In [61]:
#colocaré nueva variable en su lugar
ejec_pp['TIPO_TRANSACCION_NOMBRE'] = ejec_pp['TIPO_TRANSACCION_NOMBRE_2']

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
#borrando antigua variable
del ejec_pp['TIPO_TRANSACCION_NOMBRE_2']

### *Respuesta D: nueva variable de tipo de transacción creada con gasto y propina* 

In [63]:
ejec_pp['TIPO_TRANSACCION_NOMBRE'].value_counts()

GASTO      35204
PROPINA      430
Name: TIPO_TRANSACCION_NOMBRE, dtype: int64

### E) Crear una variable “avance_ejec” que estime el devengado con respecto al PIA (que son los recursos asignados)

### *Respuesta E: mi variable avance_ejec se expresará en porcentaje en relación entre el devengado y el pia*

In [64]:
ejec_pp['avance_ejec'] = round((ejec_pp['MONTO_DEVENGADO'] / ejec_pp['MONTO_PIA'])*100)

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
ejec_pp[['MONTO_DEVENGADO', 'MONTO_PIA', 'avance_ejec']].head()

,MONTO_DEVENGADO,MONTO_PIA,avance_ejec
10,0.00,300,0.0
40,0.00,116667,0.0
41,0.00,46019,0.0
42,0.00,545,0.0
43,2343.66,4350,54.0


In [66]:
ejec_pp.avance_ejec.value_counts() #valores de nuestra nueva variable: casi todos en 0.

0.0       27531
100.0       525
10.0        313
8.0         289
inf         286
7.0         284
9.0         259
6.0         238
12.0        236
11.0        231
4.0         202
3.0         184
1.0         183
5.0         182
13.0        176
14.0        176
15.0        167
17.0        158
2.0         155
16.0        151
19.0        124
18.0        119
21.0        109
20.0        105
22.0        105
25.0         96
27.0         89
28.0         85
23.0         82
24.0         72
          ...  
159.0         1
664.0         1
104.0         1
260.0         1
251.0         1
811.0         1
461.0         1
109.0         1
124.0         1
146.0         1
245.0         1
123.0         1
118.0         1
135.0         1
598.0         1
117.0         1
740.0         1
1480.0        1
185.0         1
181.0         1
106.0         1
182.0         1
339.0         1
718.0         1
101.0         1
520.0         1
174.0         1
126.0         1
1203.0        1
1279.0        1
Name: avance_ejec, Lengt

### F) Mostrar en una tabla el devengado de marzo y hasta marzo; así como el avance en ejecución por región y “tipo”.

In [67]:
print("Devengado de Marzo:",round(e_marzo))
print("devengado hasta Marzo:", round(ejec_pp.MONTO_DEVENGADO.sum()))

Devengado de Marzo: 3595503184.0
devengado hasta Marzo: 143699078.0


segunda parte: Agrupando por regiones

In [68]:
regiones = ejec_pp.groupby(['PLIEGO_NOMBRE'])

In [69]:
regiones['avance_ejec'].describe() #los numoers son mayoritariamente inf o 0 xq la mayoria no cuenta con devengado

,count,mean,std,min,25%,50%,75%,max
PLIEGO_NOMBRE,,,,,,,,
GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO,447.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS,900.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2718.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC,1603.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA,1382.0,inf,NaN,0.0,0.0,0.0,6.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO,1839.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA,2073.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,2599.0,inf,NaN,0.0,0.0,0.0,0.0,inf
GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVELICA,1447.0,inf,NaN,0.0,0.0,0.0,0.0,inf


tercera parte: Agrupando por tipo

In [70]:
tipo = ejec_pp.groupby(['TIPO_TRANSACCION_NOMBRE'])

In [71]:
tipo['avance_ejec'].describe() # lo mismo para este caso.

,count,mean,std,min,25%,50%,75%,max
TIPO_TRANSACCION_NOMBRE,,,,,,,,
GASTO,34618.0,inf,NaN,0.0,0.0,0.0,0.0,inf
PROPINA,418.0,inf,NaN,0.0,0.0,0.5,8.0,inf


### Respuesta F: LA RAZON por la existen tantos criterios en 0 en nuestras tablas resumen es xq la mayoria de datos se encuentran en cero (como puedes ver en el conteo de abajo: 28,023 de 35,634 casos).

In [72]:
count = 0
for x in ejec_pp.MONTO_DEVENGADO:
        if x == 0:
            count = count+1
print("devengados en 0:", count)

devengados en 0: 28023


In [73]:
ejec_pp.MONTO_DEVENGADO.count()

35634

### G) Con la base “Costos” fusionar la base “ejec_pp” y calcular lo siguiente (formula) Y categorizar a nivel de gobierno regional y “tipo” si es baja, media o satisfactoria la ejecución considerando el siguiente criterio.

In [74]:
#traer la base de datos de costo
costos = pd.read_excel("costos.xlsx")

In [75]:
costos.head(2)

,PLIEGO,UNIDAD EJECUTORA,PROGRAMA PRESUPUESTAL,PRODUCTO,ACTIVIDAD,FUNCIÓN,DIVISION FUNCIONAL,GRUPO FUNCIONAL,FUENTE,generica,...,Costo abril,Costo mayo,Costo junio,Costo julio,Costo agosto,Costo septiembre,Costo octubre,Costo noviembre,Costo diciembre,Costo Programado Anual
0,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,308. EDUCACION HUAMANGA,0090. LOGROS DE APRENDIZAJE DE ESTUDIANTES DE ...,3000001. ACCIONES COMUNES,5000276. GESTION DEL PROGRAMA,22. EDUCACION,006. GESTION,0008. ASESORAMIENTO Y APOYO,1. RECURSOS ORDINARIOS,3. BIENES Y SERVICIOS,...,0,0,0,600,0,0,0,0,600,1200
1,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,301. EDUCACION CHINCHA,0090. LOGROS DE APRENDIZAJE DE ESTUDIANTES DE ...,3000001. ACCIONES COMUNES,5000276. GESTION DEL PROGRAMA,22. EDUCACION,006. GESTION,0008. ASESORAMIENTO Y APOYO,1. RECURSOS ORDINARIOS,3. BIENES Y SERVICIOS,...,0,0,0,0,0,0,0,0,0,6820


In [76]:
#agrupar los costos programados por regiones
regiones_costos = costos.groupby(['PLIEGO'])

In [77]:
#retirar numeros y puntos
costos['PLIEGO'] = costos['PLIEGO'].str.replace('\d+', '')
costos['PLIEGO'] = costos['PLIEGO'].str.replace('.', '')

In [78]:
regiones_costos['Costo Programado Anual'].sum()

PLIEGO
 GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO     5633240
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS                 28189764
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH                   34969190
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC                 25366115
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA                 25779296
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO                 28759510
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA                54821828
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO                    55266949
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVELICA             30457993
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO                  30737888
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA                      12219021
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN                    33003753
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA LIBERTAD              25966773
 GOBIERNO REGIONAL DEL DEPARTAMENTO DE LAMBA

AHORA IRE POR EL DEVENGADO DE LAS REGIONES 

In [79]:
regiones.MONTO_DEVENGADO.sum()

PLIEGO_NOMBRE
GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO     5472343.40
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS                  4493321.01
GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH                    9318732.86
GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC                  3792859.87
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA                  7373690.23
GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO                  5736085.00
GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA                11101709.74
GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO                    12279651.20
GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVELICA              5681189.73
GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO                   7006303.98
GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA                       4114557.29
GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN                     6700649.84
GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA LIBERTAD               6084750.80
GOBIERNO RE

In [80]:
df1 = regiones.MONTO_DEVENGADO.sum().rename_axis('names').reset_index(name='devengado')

In [81]:
data_regiones = df1.sort_values(by='names')
data_regiones

,names,devengado
0,GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCION...,5472343.40
1,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS,4493321.01
2,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,9318732.86
3,GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC,3792859.87
4,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA,7373690.23
5,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO,5736085.00
6,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA,11101709.74
7,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,12279651.20
8,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVE...,5681189.73
9,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO,7006303.98


In [82]:
df2 = regiones_costos['Costo Programado Anual'].sum().rename_axis('names').reset_index(name='costos')

In [83]:
data_costos = df2.sort_values(by='names')
data_costos

,names,costos
0,GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIO...,5633240
1,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS,28189764
2,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,34969190
3,GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC,25366115
4,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA,25779296
5,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO,28759510
6,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA,54821828
7,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,55266949
8,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAV...,30457993
9,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO,30737888


In [84]:
#ya que están ordenadas alafabeticamente, podemos simplemente juntar nuestras bases de datos
data_regiones['costos'] = data_costos['costos']

In [85]:
#ahora voy a crear el porcentaje de ejecución
data_regiones['ejecución'] = (data_regiones['devengado']/data_regiones['costos'])*100
data_regiones

,names,devengado,costos,ejecución
0,GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCION...,5472343.40,5633240,97.143800
1,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS,4493321.01,28189764,15.939548
2,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,9318732.86,34969190,26.648409
3,GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC,3792859.87,25366115,14.952467
4,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA,7373690.23,25779296,28.603148
5,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO,5736085.00,28759510,19.945003
6,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA,11101709.74,54821828,20.250528
7,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,12279651.20,55266949,22.218797
8,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVE...,5681189.73,30457993,18.652541
9,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO,7006303.98,30737888,22.793707


In [86]:
#ahora voy a crear la variable categoria donde estarán nuestra ordinal.
lista_regiones = []
for x in data_regiones.ejecución:
    if x < 60:
        lista_regiones.append("Baja Ejecución")
    elif x > 80:
        lista_regiones.append("Ejecución satisfactoria")
    else:
        lista_regiones.append("Ejecución media")
        

data_regiones['categoría'] = lista_regiones

In [87]:
data_regiones

,names,devengado,costos,ejecución,categoría
0,GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCION...,5472343.40,5633240,97.143800,Ejecución satisfactoria
1,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMAZONAS,4493321.01,28189764,15.939548,Baja Ejecución
2,GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,9318732.86,34969190,26.648409,Baja Ejecución
3,GOBIERNO REGIONAL DEL DEPARTAMENTO DE APURIMAC,3792859.87,25366115,14.952467,Baja Ejecución
4,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA,7373690.23,25779296,28.603148,Baja Ejecución
5,GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO,5736085.00,28759510,19.945003,Baja Ejecución
6,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA,11101709.74,54821828,20.250528,Baja Ejecución
7,GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,12279651.20,55266949,22.218797,Baja Ejecución
8,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVE...,5681189.73,30457993,18.652541,Baja Ejecución
9,GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANUCO,7006303.98,30737888,22.793707,Baja Ejecución


## Considerando el último resultado, que me puede decir de la ejecución de los gobiernos regionales?

### *En base a la data adquirida, la región Callao es una excepción a la regla dentro de el panorama de bajo ejecución de los gobiernos regionales. Una de esa naturaleza también me daría cierta suspicacia sobre la calidad de esa ejecución considerando el mes de la envaluación. Por otro lados, nuevamente, tomando en cuenta el mes de evaluación, es posible que los criterios de medición para una ejecución satisfactoria sean un poco exigentes.* 